<a href="https://colab.research.google.com/github/jiansim03/colab/blob/main/7_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# im2col

# coding: utf-8
import numpy as np


def smooth_curve(x):
    """손실 함수의 그래프를 매끄럽게 하기 위해 사용

    참고：http://glowingpython.blogspot.jp/2012/02/convolution-with-numpy.html
    """
    window_len = 11
    s = np.r_[x[window_len-1:0:-1], x, x[-1:-window_len:-1]]
    w = np.kaiser(window_len, 2)
    y = np.convolve(w/w.sum(), s, mode='valid')
    return y[5:len(y)-5]


def shuffle_dataset(x, t):
    """데이터셋을 뒤섞는다.

    Parameters
    ----------
    x : 훈련 데이터
    t : 정답 레이블

    Returns
    -------
    x, t : 뒤섞은 훈련 데이터와 정답 레이블
    """
    permutation = np.random.permutation(x.shape[0])
    x = x[permutation,:] if x.ndim == 2 else x[permutation,:,:,:]
    t = t[permutation]

    return x, t

def conv_output_size(input_size, filter_size, stride=1, pad=0):
    return (input_size + 2*pad - filter_size) / stride + 1


def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).

    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩

    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """(im2col과 반대) 2차원 배열을 입력받아 다수의 이미지 묶음으로 변환한다.

    Parameters
    ----------
    col : 2차원 배열(입력 데이터)
    input_shape : 원래 이미지 데이터의 형상（예：(10, 1, 28, 28)）
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩

    Returns
    -------
    img : 변환된 이미지들
    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

In [9]:
x1 = np.random.rand(1, 3, 7, 7) # 입력데이터 (데이터수, 채널수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
col1.shape

(9, 75)

In [10]:
x2 = np.random.rand(10, 3, 7, 7) # 입력데이터 (데이터수, 채널수, 높이, 너비)
col2 = im2col(x2, 5, 5, stride=2, pad=0)
col2.shape

(40, 75)

### Affine 계층 구현
완전연결계층 구현

In [11]:
class Affine:
  def __init__(self, w, b) : # 초기화
    self.W = w
    self.b =b
    self.x = None
    self.dW = None
    self.db = None

  def forward(self, x): # 순전파
    self.x = x
    out = np.dot(x, self.w) + self.b # 행렬곱 x * 가중치 + 편향

    return out

  def backward(self, dout): # dout -> 업스트림
    dx = np.dot(dout, self.W.T) # 역전파의 곱 : 순전파 입력 신호를 서로 바꾼값을 곱
    self.dW = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis=0)

    return dx

### 소프트맥스 계층

In [13]:
def Softmax(a):
  exp_a = np.exp(a)
  sum_exp_a = np.sum(exp_a)
  y=exp_a / sum(exp_a)

  return y

### cross entropy 계층

In [14]:
def cross_entropy_error(y, t):
  delta = 1e-7
  return -np.sum(t * np.log ( y + delta ))

### Softmax-with-Loss 계층
정답레이블이 여러개일때의 손실함수  
각 원소가 정답일 확률(소프트맥스)과 정답에 대한 차를 레이블 각각에 대해 구한다

In [15]:
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None # 손실
    self.y = None # softmax의 출력
    self.t = None # 정답레이블 (원-핫)

  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)
    return self.loss

  def backward(self, dout=1):
    batch_size = self.t.shape
    dx = (self.y - self.t) / batch_size

    return dx




### 합성곱계층 구현하기

In [16]:
class Convolution:
  def __init__(self, W, b, stride=1, pad=0):
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad

  def forward(self, x): # 순전파
    FN, C, FH, FW = self.W.shape # 필터개수, 채널, 필터 높이, 필터 너비를 인풋의 형태로
    N, C, H, W = x.shape # 갯수, 채널, 높이, 너비
    out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
    out_w = int(1 + (W + 2*self.pad - FW) / self.stride)

    col = im2col(x, FH, FW, self.stride, self.pad)
    col_W = self.W.reshape(FN, -1).T # 필터 전개
    out = np.dot(col, col_W) + self.b # 이미지 * 필터

    out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

    return out

### 풀링계층 구현하기
1. 입력데이터 전개하기
2. 행(윈도우)별 최댓값 구하기
3. 적절히 성형해주기

In [17]:
class Pooling:
  def __init__(self, pool_h, pool_w, stride=1, pad=0):
    self.pool_h = pool_h
    self.pool_w = pool_w
    self.stride = stride
    self.pad = pad

  def forward(self, x):
    N, C, H, W = x.shape
    out_h = int(1 + (H - self.pool_h) / self.stride)
    out_w = int(1 + (W - self.pool_w) / self.stride)

    # 전개 (1)
    col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
    col = col.reshpe(-1, self.pool_h*self.pool_w)

    # 최댓값 (2)
    out = np.max(col, axis=1)

    # 성형 (3)
    out = out.reshape(N, out_h, out_w, C).transposse(0,3,1,2)

    return out

### CNN 구현하기

In [69]:
# SimpleConvNet초기화
# 1.1. 초기화 인수로 주어진 컨볼루션 계층의 하이퍼 파라미터를 딕셔너리에 꺼낸다.
# 1.2. 컨볼루션 계층의 출력크기를 계산한다

from collections import OrderedDict  # collections 모듈 임포트

class SimpleConvNet:
  def __init__(self, input_dim=(1, 28, 28),
               conv_param={'filter_num' :30, 'filter_size':5, 'pad' : 0, 'stride':1},
               hidden_size=100, output_size=10, weight_init_std=0.01) :

    filter_num = conv_param['filter_num']
    filter_size = conv_param['filter_size']
    filter_pad = conv_param['pad']
    filter_stride = conv_param['stride']
    intput_size = input_dim[1]
    conv_output_size = (intput_size - filter_size + 2 * filter_pad) / filter_stride + 1 # 컨볼루션 후 사이즈
    pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

    # --------- 가중치 매개변수 초기화 ---------
    # 1(CNN), 2,3(어파인) 계층의 매개변수를 초기화한다
    self.params = {}
    # 1번째(CNN) 계층의 가중치, 편향
    self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
    self.params['b1'] = np.zeros(filter_num)
    # 2번째(affine) 계층의 가중치, 편향
    self.params['W2'] = weight_init_std * np.random.randn(pool_output_size,hidden_size)
    self.params['b2'] = np.zeros(hidden_size)
    # 2번째(affine) 계층의 가중치, 편향
    self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b3'] = np.zeros(output_size)


# CNN 구성하는 계층들 생성

    self.layers = OrderedDict() # 딕셔너리의 아이템이 삽입된 순서대로 유지
    self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
    self.layers['Relu1'] = Relu()
    self.layers['Pool1'] = Pooling(pool_h = 2, Pool_w =2, stride=2)
    self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
    self.layers['Relu2'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
    self.last_layer = SoftmaxWithLoss()

  # 추론
  def predict(self, x):
    for layer in self.layers.values(): # 첫번째 레이어부터 순차적으로 수행하며 결과를 x에 할당 후 다음 레이어 전달
      x = layer.forward
    return x

  # 손실함수
  def loss(self, x, t):
    y = self.predit(x)
    return self.last_layer.forward(y,t) # 정답과 예측의 오차 구해 확률 구한다(SoftmaxWithLoss)

In [70]:
# 오차역전파법으로 기울기 구하기

def gradient(self, x, t):
  # 순전파
  self.loss(x,t)

  # 역전파
  dout = 1
  dout = self.last_layer.backward(dout)  # 상류 값을 역전파 함수에 넣은 결과를 다시 dout에 넣는다

  layers = list(self, layers. values()) # 각 레이어 딕셔너리의 밸류값을 한 리스트에 넣는다
  layers.reverse() # 레이어의 순서를 뒤짚는다
  for layer in layers: # 레이어 리스트 갯수만큼, 레이어의 백워드 함수에 상류의 값을 넣은 것을 dout에 재할당한다
    dout = layer.backward(dout)

  # 결과 저장
  grads = {}
  grads['W1'] = self.layers['Conv1'].dW
  grads['b1'] = self.layers['Conv1'].db
  grads['W2'] = self.layers['Affine1'].dW
  grads['b2'] = self.layers['Affine1'].db
  grads['W3'] = self.layers['Affine2'].dW
  grads['b3'] = self.layers['Affine2'].db

  return grads